# Regularisierung auf dem Titanic-Datensatz

Hinweis: Da das Notebook nur das Prinizip der Regularisierung zeigen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (d.h. kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [21]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]
df_selection = df_selection.dropna()
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [23]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

## Aufgaben:

### Aufgabe 1: Ridge-Regularisierung
1. Trainieren Sie eine Logistische Regression mit Ridge-Regularisierung (siehe [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html)) mit $\alpha =100$ auf den Trainingsdaten.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [24]:
from sklearn.linear_model import RidgeClassifier
ridge = RidgeClassifier(alpha=100, max_iter=1000)
ridge.fit(X_train, y_train)
pred = ridge.predict(X_test)

from sklearn.metrics import accuracy_score
print("Accuracy: " + str(accuracy_score(y_test, pred)))
print(pd.Series(ridge.coef_[0], index = df_X.columns))
print("Summe der absoluten Gewichte " + str(sum(np.absolute(ridge.coef_[0]))))

Accuracy: 0.6713286713286714
Pclass   -0.320888
Age      -0.013431
SibSp    -0.096943
Parch     0.092842
Fare      0.002719
dtype: float64
Summe der absoluten Gewichte 0.5268232381956689


### Aufgabe 2: Cross-Validation über $\alpha$
1. Führen Sie nun eine Cross-Validation mit Hilfe der Klasse [RidgeClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html) durch. Benutzen Sie dafür die gleichen Werte für $\alpha$ wie in [diesem Notebook](https://github.com/pabair/ml-kurs/blob/master/4_Regularisierung%2BCV.ipynb) in Zelle 9.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [25]:
from sklearn.linear_model import RidgeClassifierCV

alphas = range(1,10000)
ridgecv = RidgeClassifierCV(alphas = alphas)
ridgecv.fit(X_train, y_train)
pred = ridgecv.predict(X_test)
print("Alpha: " + str(ridgecv.alpha_))
print("Accuracy: " + str(accuracy_score(y_test, pred)))

Alpha: 6
Accuracy: 0.6853146853146853


### Aufgabe 3:  Lasso-Regularisierung
Für die Lasso-Regularisierung gibt es keine extra Unterklasse, da diese bei der Klasse `LogisitcRegression` über den Parameter `penalty` eingestellt werden kann. Mit dem Parameter `penalty = "l1"` wird die Lasso-Regression verwendet.
Der Regularisierungsfaktor $\alpha$ wird in diesem Fall über den Parameter `C` bestimmt, welcher den Default-Wert `C=1.0` hat:

`LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")`

Beachten Sie dabei:
- Für die Regression ist das zusätzliche Argument `solver="liblinear"` nötig, da der Standard-Optimierer nicht mit `L1` funktioniert.
- Parameter `C` gibt die <ins>inverse</ins> Stärke der Regularisierung an (Details siehe [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).


1. Trainieren Sie eine Logistische Regression mit Lasso-Regression für verschiedene Parameter `C`:
    - den Default-Wert `C=1.0`.
    - wählen Sie `C` so, dass das Modell sehr stark regularisiert ist.
    - wählen Sie `C` so, dass das Modell keine Regularisierung hat.
2. Vergleiche Sie die Ergebnisse dieser Modelle indem Sie:
    - die Gewichte und deren Summe ausgeben (wie in [diesem Notebook](https://github.com/pabair/ml-kurs/blob/master/4_Regularisierung%2BCV.ipynb)).
    - die Accuracy auf den Testdaten ermitteln. 

In [26]:
lasso = LogisticRegression(max_iter=1000, penalty = "l1", C= 1.0, solver="liblinear")
lasso_strongReg = LogisticRegression(max_iter=1000, penalty = "l1", C= 0.0001, solver="liblinear")
lasso_weakReg = LogisticRegression(max_iter=1000, penalty = "l1", C= 1000.0, solver="liblinear")

lasso.fit(X_train, y_train)
lasso_strongReg.fit(X_train, y_train)
lasso_weakReg.fit(X_train, y_train)

pred1 = lasso.predict(X_test)
pred2 = lasso_strongReg.predict(X_test)
pred3 = lasso_weakReg.predict(X_test)

print("Accuracy normal: " + str(accuracy_score(y_test, pred1)))
print("Gewichte der Features:")
print(pd.Series(lasso.coef_[0], index = df_X.columns))
print("Summe der absoluten Gewichte " + str(sum(np.absolute(lasso.coef_[0]))))
print("")

print("Accuracy strong: " + str(accuracy_score(y_test, pred2)))
print("Gewichte der Features:")
print(pd.Series(lasso_strongReg.coef_[0], index = df_X.columns))
print("Summe der absoluten Gewichte " + str(sum(np.absolute(lasso_strongReg.coef_[0]))))
print("")

print("Accuracy weak: " + str(accuracy_score(y_test, pred3)))
print("Gewichte der Features:")
print(pd.Series(lasso_weakReg.coef_[0], index = df_X.columns))
print("Summe der absoluten Gewichte " + str(sum(np.absolute(lasso_weakReg.coef_[0]))))
print("")

Accuracy normal: 0.6923076923076923
Gewichte der Features:
Pclass   -0.936400
Age      -0.037640
SibSp    -0.308794
Parch     0.298053
Fare      0.007302
dtype: float64
Summe der absoluten Gewichte 1.5881890748436096

Accuracy strong: 0.5524475524475524
Gewichte der Features:
Pclass    0.0
Age       0.0
SibSp     0.0
Parch     0.0
Fare      0.0
dtype: float64
Summe der absoluten Gewichte 0.0

Accuracy weak: 0.6853146853146853
Gewichte der Features:
Pclass   -1.052985
Age      -0.041775
SibSp    -0.336393
Parch     0.327286
Fare      0.005949
dtype: float64
Summe der absoluten Gewichte 1.7643873042701608



### Aufgabe 4. Cross-Validierung über  `C`
Ein guter Wert für `C` soll nun mit Hilfe der Methode `cross_val_score` gefunden werden. Ein Beispiel für diese Methode ist:

In [27]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
accuracies = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
print(accuracies)

[0.65217391 0.73684211 0.6754386  0.71929825 0.71052632]


Die Methode führt eine k-fold Cross-Validation auf den Trainingsdaten durch und gibt für jeden der k-folds den Score (in diesem Fall "accuracy") des Modell auf den jeweiligen Validierungsdaten zurück.

1. Berechnen Sie den Durchschnitt der Rückgabewerte von `cross_val_score` um für das gegebene `model` den durchschnittlichen Accuarcy-Score über alle k-folds zu erhalten.

In [28]:
from numpy import mean

mean_accuracy = mean(accuracies)
mean_accuracy

0.6988558352402746

2. Berechnen Sie für jeden der Werte $ C \in [0.1, 0.2, ...,9.8, 9.9]$ den durchschnittlichen Accuarcy-Score.

In [29]:
mean_accuracies = []

for c in range(1, 100):
    model = LogisticRegression(max_iter=1000, penalty = "l1", C=c/10., solver="liblinear")
    accuracies = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    mean_accuracy = mean(accuracies)
    mean_accuracies.append(mean_accuracy)

print(mean(mean_accuracies))

0.6953151653838151


3. Trainieren Sie das Modell mit dem besten Wert für $C$ auf allen Trainingsdaten.

In [30]:
max_accuracy = max(mean_accuracies)
print("Max. Accur: " + str(max_accuracy))
max_index = np.argmax(mean_accuracies)
max_C = (max_index+1) / 10
max_C

Max. Accur: 0.7216475972540046


0.3

4. Bestimmen Sie die Accuracy dieses Modells auf den Testdaten.

In [31]:
model = LogisticRegression(max_iter=1000, penalty = "l1", C=max_C, solver="liblinear")
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("Accuracy : " + str(accuracy_score(y_test, pred)))
#0.6923076923076923
#0.6923076923076923

Accuracy : 0.6923076923076923
